# 07.01.24 Analyse von Much Ado About Nothing von Shakespeare <hr>

### TODO
- Textquelle zitieren
- Text lesen
- Analyse Fragen finden

### Mögliche Analysen:
- Charaktere Redeanteil vergleichen
- Welche Wörter werden von bestimmten Charakteren verwendet
- Alte Sprache anschauen, wie genau noch unbekannt: Liste mit Wörtern, die in modernen Texten nicht vorhanden sind
- Gefühlsanalyse der Charaktere mit Mood-Analyse 
- Wiederkehrende Motive analysieren? Metaphern, mit Word-Patterns
- Sprachunterschiede basierend auf dem Geschlecht?
- Unterschied zwischen prose `<ab>` und verse `<l>`
#### Interessanteste Punkte:
- Mood Analyse:
    - 
- Geschlechterrollen sprachlich analysieren:
    - Redeanteile der Charaktere vergleichen: Beatrice - Benedick und Claudio - Hero
    - Welche Wörter werden von bestimmten Charakteren verwendet 
        - maßzahlen zeta, usw. gr. Buchstaben, inhaltsvokabeln
    - Gibt es gemeinsamkeiten bei den Geschlechtern?
    - Länge der Sätze der Charaktere, was ist das maximum minimum, median
    - Kein Komplettes Projekt, was könnte man in der Zukunft noch machen, z.B. Plotten, ka.
    - Interpretationsmöglichkeiten, Literarisch, welche Disziplinen/Aspekte der Linguistik, Literaturwissenschaften wären interessant

TEI Klassen am Anfang gelöscht um Komplikationen zu vermeiden
Zuerst einen zusammenhängenden Lesbaren Text erstellt. Schwierigkeiten mit `<lb>`Elementen

In [ ]:
from lxml import etree
from collections import Counter
import statistics
# Import numpy for debugging
import numpy as np

# Parse the XML file --> Download Much Ado About Nothing von DraCor (https://www.dracor.org)
tree_text = etree.parse(r"Shakespeare_MuchAdoAboutNothing_DraCor.xml")

# Specify only parts of speech
speech_elements = tree_text.xpath("//sp")

# Count the number of words per character
speech_counts = Counter()

# List to store all words
all_spoken_words = []

# Dictionary to store all spoken text per character
spoken_by_character = {}

# List to store all words spoken by Beatrice, Benedick, Claudio and Hero
target_characters = ["#Beatrice_Ado", "#Benedick_Ado", "#Claudio_Ado", "#Hero_Ado"]

# Creation of a readable text from the complex xml file
for sp in speech_elements:
    # Select <w> words, <pc> punctuation, <lb> line breaks, and <c> spaces elements inside <p> elements
    spoken_elements = sp.xpath(".//p//*[self::w or self::pc or self::lb or self::c]")

    for elem in spoken_elements:
        if elem.tag == "lb" or elem.tag == "c":
            # Replace <lb>  and <c> with a space
            all_spoken_words.append(" ")
        elif elem.tag == "pc":
            # Attach punctuation directly to the previous word
            if all_spoken_words and all_spoken_words[-1] != " ":
                all_spoken_words[-1] = all_spoken_words[-1].rstrip() + elem.text
                
        else:
            # Add a word followed by a space
            all_spoken_words.append(elem.text + " ")

for sp in speech_elements:
    # Get the speaker from the 'who' attribute
    speaker = sp.get("who")
    # Check if speaker exists
    if speaker in target_characters:
        # Simplify the speaker name by removing the "#" and "_Ado" parts
        simple_name = speaker.strip("#").replace("_Ado", "")
        # Get all words spoken by the character within the speech element and add the count of words to the character
        words = sp.xpath(".//p//w/text()")
        speech_counts[simple_name] += len(words)
        # Store the text for sentence length analysis
        if simple_name not in spoken_by_character:
            spoken_by_character[simple_name] = []
        spoken_by_character[simple_name].append(" ".join(words))

# Calculate average sentence length
def calculate_average_sentence_length(texts):
    sentences = []
    for text in texts:
        # Replace "!" and "?" and ":" with ".". Split the text into sentences at ".". Remove leading/trailing spaces and filter out empty strings 
        sentences.extend([s.strip() for s in text.replace("!", ".").replace(":",".").replace("?",".").split(".") if s.strip()])

    # Handle the case where there are no sentences
    if not sentences:
        return 0, 0, 0, 0, 0
    
    # Count the number of words in each sentence
    sentence_lengths = [len(sentence.split()) for sentence in sentences]
    
    # Calculate the average, maximum, minimum,  median and mode sentence lengths
    # np.argmax used for debugging
    avg_length = sum(sentence_lengths) / len(sentence_lengths)
    max_length = max(sentence_lengths)
    max_index = np.argmax(sentence_lengths)
    min_length = min(sentence_lengths)
    med_length = statistics.median(sentence_lengths)
    mode_length = statistics.mode(sentence_lengths)
    
    return avg_length, max_length, min_length, med_length, mode_length, sentences[max_index]
    
# Compute average sentence length for each character
sentence_lengths = {
    character: calculate_average_sentence_length(spoken_by_character[character])
    for character in spoken_by_character
}
# Print the speech counts
print("Speech Counts by Character:")
for speaker, count in speech_counts.items():
    print(f"{speaker}: {count} words")

# Print the average sentence length for each character
print("\nAverage sentence length by character:")
for character, length in sentence_lengths.items():
    avg_length, max_length, min_length, med_length, mode_length, sentence = length
    print(f"{character}: Avg: {avg_length:.2f} words, Med: {med_length:} words\n Max: {max_length} words, Min: {min_length} words")
    print(f"{character}: Mode: {mode_length}")

    print(sentence)
# Combine spoken words into a single string, removing unnecessary spaces
spoken_text = "".join(all_spoken_words).strip()
# Remove extra spaces introduced by line breaks or punctuation
spoken_text = " ".join(spoken_text.split())

print(spoken_text)


Speech Counts by Character:
Beatrice: 2186 words
Hero: 188 words
Benedick: 3445 words
Claudio: 1023 words

Average sentence length by character:
Beatrice: Avg: 23.26 words, Med: 15.5 words
 Max: 102 words, Min: 2 words
Beatrice: Mode: 8
The fault will be in the music cousin if you be not wooed in good time If the Prince be too important tell him there is measure in everything and so dance out the answer For hear me Hero wooing wedding and repenting is as a Scotch jig a measure and a cinquepace The first suit is hot and hasty like a Scotch jig and full as fantastical the wedding mannerly modest as a measure full of state and ancientry and then comes repentance and with his bad legs falls into the cinquepace faster and faster till he sink into his grave
Hero: Avg: 8.95 words, Med: 7 words
 Max: 22 words, Min: 2 words
Hero: Mode: 7
So you walk softly and look sweetly and say nothing I am yours for the walk and especially when I walk away
Benedick: Avg: 28.95 words, Med: 14 words
 Max: 310

# Sentiment Analysis: 
---
### work in progress

In [ ]:
from lxml import etree
from collections import Counter
from textblob import TextBlob

# Parse the XML file --> Download Much Ado About Nothing von DraCor (https://www.dracor.org)
tree_text = etree.parse(r"Shakespeare_MuchAdoAboutNothing_DraCor.xml")

# Specify only parts of speech
speech_elements = tree_text.xpath("//sp")

# Dictionary to store all spoken words per character
spoken_by_character = {}

# List to store all speeches for overall sentiment analysis
all_speeches = []

for sp in speech_elements:
    # Get the speaker from the 'who' attribute
    speaker = sp.get("who")
    if not speaker:
        continue
    
    # Select words, punctuation, and <lb>, <c> elements
    spoken_elements = sp.xpath(".//p//*[self::w or self::pc or self::lb or self::c]")

    # Construct the full speech text
    speech_words = []
    for elem in spoken_elements:
        if elem.tag in ["lb", "c"]:
            speech_words.append(" ")
        elif elem.tag == "pc":
            if speech_words:
                speech_words[-1] = speech_words[-1].rstrip() + elem.text
        else:
            speech_words.append(elem.text + " ")
    
    # Combine words into a single speech
    full_speech = "".join(speech_words).strip()
    
    # Store speech for sentiment analysis
    all_speeches.append(full_speech)
    
    # Append to the speaker's dialogue
    if speaker not in spoken_by_character:
        spoken_by_character[speaker] = []
    spoken_by_character[speaker].append(full_speech)

# Perform sentiment analysis
overall_sentiment = TextBlob(" ".join(all_speeches)).sentiment
print(f"Overall Sentiment of the Play: Polarity={overall_sentiment.polarity}, Subjectivity={overall_sentiment.subjectivity}")

# Analyze sentiment for each character
character_sentiments = {}
for character, speeches in spoken_by_character.items():
    combined_speeches = " ".join(speeches)
    sentiment = TextBlob(combined_speeches).sentiment
    character_sentiments[character] = sentiment
    print(f"Character: {character}, Polarity={sentiment.polarity}, Subjectivity={sentiment.subjectivity}")


Overall Sentiment of the Play: Polarity=0.205172478709844, Subjectivity=0.5589000239764765
Character: #Leonato_Ado, Polarity=0.20844516594516585, Subjectivity=0.5515476190476191
Character: #MESSENGERS.LEONATO.1_Ado, Polarity=0.34245238095238095, Subjectivity=0.4961190476190477
Character: #Beatrice_Ado, Polarity=0.14516392864764344, Subjectivity=0.5697335528321441
Character: #Hero_Ado, Polarity=0.40535714285714286, Subjectivity=0.6223214285714286
Character: #DonPedro_Ado, Polarity=0.25688146010368235, Subjectivity=0.5732422371866817
Character: #Benedick_Ado, Polarity=0.22820487100974912, Subjectivity=0.5715827115522237
Character: #DonJohn_Ado, Polarity=0.19897759103641458, Subjectivity=0.5387052598817306
Character: #Claudio_Ado, Polarity=0.24938653470647518, Subjectivity=0.5217724867724868
Character: #LeonatosBrother_Ado, Polarity=0.27549999999999997, Subjectivity=0.4656666666666666
Character: #Conrade_Ado, Polarity=0.11218828914141414, Subjectivity=0.6712121212121213
Character: #Borach